In [1]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [1]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../src/')

In [2]:
import utils, split, merge, document_analysis
from split import split_merge

import logging
import os
import math
import io
import json
import PyPDF2

logging.getLogger().setLevel(logging.INFO)

In [3]:
def run_example(page_limit, size_limit):
    try:
        # select a smaller page limit for this toy example
        current_async_page_limit = page_limit
        current_async_size_limit = size_limit
        
        # specify the file name as it appears in s3
        file_name = "test_example1.pdf"
        input_file = io.BytesIO(utils.read_file_from_s3(file_name))

        if file_name.split(".")[1] == "pdf":
            reader = PyPDF2.PdfFileReader(input_file)
            textract_response = {}
            while True:
                current_async_page_limit, textract_response = split_merge(
                    file_name, reader, current_async_page_limit, current_async_size_limit
                )
                if len(textract_response) > 0:
                    return textract_response
        else:
            raise ValueError("Expected a PDF file as input")
    except Exception as e:
        print(e)

Toy example with textract page limit as 5

In [4]:
%%timeit -r 1 -n 1
textract_response = run_example(5, 524288000)
document_analysis.pretty_print_text(textract_response)

INFO:root:Started job with id: eeca89430eec8a0131f8046b14e426efa193484439f82da1ca25f1d7bf41ee94
INFO:root:Job status: IN_PROGRESS
INFO:root:Job status: SUCCEEDED
INFO:root:Started job with id: 0d6919831ad5d611b24822dbd53216c97cfa415ba6c6027c0fd52d0592248949
INFO:root:Job status: SUCCEEDED


On page 1
On page 2
On page 3
On page 4
On page 5
On page 6
On page 7
25.8 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
def run_document_detection(file_name):
    job_id = document_analysis.start_job(merge.client,utils.BUCKET,file_name)
    logging.info("Started job with id: {}".format(job_id))

    if document_analysis.is_job_complete(merge.client, job_id):
        response = document_analysis.get_job_results(merge.client, job_id)

In [6]:
def test_merge_results(merge_results, no_split_response):
    assert len(merge_results[0]["Blocks"]) == len(no_split_response[0]["Blocks"])

In [7]:
%%timeit -r 1 -n 1
no_split_response = run_document_detection("test_example1.pdf")

INFO:root:Started job with id: 64ef3808f63cd852b3733ac47d90446bde8023e69cee0f01ad263a770d540b07
INFO:root:Job status: SUCCEEDED


10.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Toy example with textract size limit as 38000 (37.1KB)

In [8]:
%%timeit -r 1 -n 1
textract_response = run_example(5, 38000)
document_analysis.pretty_print_text(textract_response)

INFO:root:File split at split_size=5 exceeded Textract's size limitation
INFO:root:File split at split_size=3 exceeded Textract's size limitation
INFO:root:Started job with id: aadd1a268667f3dfe9a24672d7cf97189db3e90824a9a8da3fd294c691f1d0a8
INFO:root:Job status: SUCCEEDED
INFO:root:Started job with id: e46c9401711bcbb7f5baf0c2798c881d6fbc8ff927f4a7c6c7a1408fd437f2da
INFO:root:Job status: SUCCEEDED
INFO:root:Started job with id: fe7aa0572b746178678d2862cf99288e4a38d3aea07b87c758614588f0cfdaab
INFO:root:Job status: SUCCEEDED
INFO:root:Started job with id: 753ad4a7b5692eaa82996b58d0187483ebf3e296fb36343462c7e4a654ddd96a
INFO:root:Job status: IN_PROGRESS
INFO:root:Job status: SUCCEEDED


On page 1
On page 2
On page 3
On page 4
On page 5
On page 6
On page 7
46.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
